# 循环神经网络(RNN)实践：预测混凝土强度随时间的变化

在这个实操练习中，我们将使用循环神经网络(RNN)来预测混凝土强度随时间的变化。这是一个时间序列预测问题，我们的模型将学习混凝土强度随时间变化的模式。

## 目标
1. 理解RNN的基本结构和工作原理
2. 学习如何使用Python和PyTorch构建简单的RNN模型
3. 实践时间序列预测任务
4. 了解模型训练和评估的过程

让我们开始吧！

## 1. 导入必要的库

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

print("PyTorch version:", torch.version)

## 2. 准备数据

为了简化演示，我们将生成一个模拟的混凝土强度时间序列数据集。在实际应用中，你需要使用真实的实验数据。

In [ ]:
# 生成模拟数据
np.random.seed(42)
time = np.arange(0, 365, 1)
strength = 20 + 10 * np.log(time + 1) + np.random.normal(0, 0.5, 365)

# plot strength with time
plt.figure(figsize=(10, 6))
plt.plot(time, strength)
plt.xlabel("Time")
plt.ylabel("Strength")
plt.title("Strength vs Time")
plt.show()

# 划分训练集和测试集
train_size = int(0.8 * len(time))
train_data = strength[:train_size]
test_data = strength[train_size:]

# 创建序列
def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+seq_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

seq_length = 7  # 使用过去7天的数据预测下一天
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# 转换为PyTorch张量
X_train = torch.FloatTensor(X_train).unsqueeze(2)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test).unsqueeze(2)
y_test = torch.FloatTensor(y_test)

# 创建DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("训练集形状:", X_train.shape)
print("测试集形状:", X_test.shape)

## 3. 构建RNN模型

现在，我们将构建一个简单的RNN模型。这个模型包含一个LSTM层和一个全连接层。

In [ ]:
class ConcreteStrengthRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ConcreteStrengthRNN, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

model = ConcreteStrengthRNN(input_size=1, hidden_size=64, output_size=1)
print(model)

## 4. 训练模型

现在我们开始训练模型。

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# config learning rate scheduler
# first 200 epochs: lr = 0.001
# next 200 epochs: lr = 0.0001
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[400, 500], gamma=0.1)

num_epochs = 500
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

losses = []
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        outputs = model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    losses.append(train_loss / len(train_loader))

    scheduler.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# 绘制损失曲线
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.show()

## 5. 评估模型性能

让我们在测试集上评估模型的性能。

In [ ]:
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        outputs = model(batch_X)
        predictions.extend(outputs.cpu().numpy().flatten())
        actuals.extend(batch_y.numpy().flatten())

predictions = np.array(predictions)
actuals = np.array(actuals)

mse = np.mean((predictions - actuals) ** 2)
print(f'测试集均方误差: {mse:.4f}')

plt.figure(figsize=(12, 6))
plt.plot(actuals, label='Actual')
plt.plot(predictions, label='Predicted')
plt.ylim(50, 100)
plt.legend()
plt.title('Concrete Strength Prediction vs Actual')
plt.xlabel('Time (days)')
plt.ylabel('Concrete Strength (MPa)')
plt.show()

## 6. 使用模型进行预测

最后，让我们使用训练好的模型对未来7天的混凝土强度进行预测。

In [ ]:
def predict_next_7_days(model, last_week_data):
    model.eval()
    predictions = []
    current_input = last_week_data.clone()
    
    for _ in range(7):
        with torch.no_grad():
            next_day_pred = model(current_input.unsqueeze(0).to(device))
        predictions.append(next_day_pred.item())
        current_input = torch.cat((current_input[1:], next_day_pred.cpu()))
    
    return predictions

last_week_data = X_test[-1]
# print(last_week_data.shape)
next_7_days_pred = predict_next_7_days(model, last_week_data)

plt.figure(figsize=(12, 6))
plt.plot(test_data, linestyle='--', label='Actual')
plt.plot(range(len(test_data), len(test_data) + 7), next_7_days_pred, marker='o', label='Predicted')
# plt.ylim(74.2, 74.8)
plt.title('Concrete Strength Prediction for Next 7 Days')
plt.xlabel('Days')
plt.ylabel('Predicted Concrete Strength (MPa)')
plt.legend(loc='upper left')
plt.show()

print("未来7天的混凝土强度预测值：")
for i, pred in enumerate(next_7_days_pred, 1):
    print(f"第{i}天: {pred:.2f} MPa")

## 总结

在这个实操练习中，我们构建了一个简单的RNN模型来预测混凝土强度随时间的变化。我们学习了如何：

1. 准备时间序列数据
2. 构建RNN模型
3. 训练模型
4. 评估模型性能
5. 使用模型进行预测

这个示例展示了RNN在土木工程中的一个实际应用。通过这种方法，我们可以预测混凝土强度随时间的变化，这对于工程项目的规划和质量控制非常有帮助。

在实际应用中，你需要使用真实的实验数据，可能还需要考虑更多的因素（如温度、湿度等）来提高预测的准确性。此外，你可以尝试使用更复杂的模型架构（如LSTM或GRU）或增加更多的特征来进一步提高模型的性能。